# California Drought Data Ingestion

**Dataset**: US Drought Monitor - California  
**Date Range**: 2000 - 2025  
**Source**: [US Drought Monitor](https://droughtmonitor.unl.edu/)

**Objective**: 
- Load and validate weekly drought severity data
- Verify data quality and temporal coverage
- Understand dataset structure and drought categories (D0-D4)
- Document drought conditions during January 2025 fires
- Prepare for comprehensive EDA in Phase 2


## Why Drought Data Matters for Fire Prediction

Drought is a long-term drying process that creates fundamental conditions for catastrophic wildfires. Unlike daily weather, drought represents sustained moisture deficits.

**1. Fuel Moisture Depletion**: Progressive drying reduces vegetation moisture over weeks/months. Severe drought means critically dry fuels.

**2. Long-Term Risk Indicator**: Drought observable weeks/months before fires, providing early warning.

**3. California Drought Cycles**: Multi-year droughts (2012-2016) preceded massive fire seasons. Mediterranean climate with natural dry summers.

**4. Cumulative Effect**: Each drought week compounds fire risk. D2+ drought = high fire danger.

**5. Synergy with Weather**: Drought + Heat + Wind = Extreme conditions (January 2025 scenario).

**Key Insight**: January 2025 fires followed months of below-normal rainfall. Tracking drought severity predicts when fuels reach critically low moisture.


### Dataset Features Explained

#### Drought Severity Categories (Categorical Features):

| Feature | Type | Values | Meaning | Fire Risk Impact |
|---------|------|--------|---------|------------------|
| **None** | Categorical | 0-100% | Percent of area with NO drought | Safe - Normal fuel moisture |
| **D0** | Categorical | 0-100% | Abnormally Dry | **Low** - Fuels starting to dry, minor stress |
| **D1** | Categorical | 0-100% | Moderate Drought | **Moderate** - Vegetation stress, reduced moisture |
| **D2** | Categorical | 0-100% | Severe Drought | **High** - Critically dry fuels, fire danger |
| **D3** | Categorical | 0-100% | Extreme Drought | **Very High** - Explosive fire potential |
| **D4** | Categorical | 0-100% | Exceptional Drought | **Extreme** - Catastrophic fire conditions |

**Value Interpretation**:
- Each value represents the **percentage of California's area** in that drought category for that week
- Example: D2 = 45% means 45% of California is experiencing Severe Drought
- Values sum to approximately 100% across all categories (None + D0 + D1 + D2 + D3 + D4)

#### Temporal Features:

| Feature | Type | Format | Meaning |
|---------|------|--------|---------|
| **ValidStart** | Date | YYYY-MM-DD | Start date of drought measurement week |
| **ValidEnd** | Date | YYYY-MM-DD | End date of drought measurement week |
| **MapDate** | Date | YYYYMMDD | Date drought map was released |

**Frequency**: Weekly measurements (every Tuesday)

#### Geographic Features:

| Feature | Type | Values | Meaning |
|---------|------|--------|---------|
| **FIPS** | Categorical | 06 | Federal Information Processing Standard code for California |
| **State** | Categorical | CA | State abbreviation |
| **County** | Categorical | Name | County name (if county-level data) |

#### Derived Features for ML Model:

From this drought data, we can create powerful predictive features:

1. **Current Drought Level** = Highest active drought category (D0-D4)
2. **Drought Intensity Score** = Weighted sum (D1×1 + D2×2 + D3×3 + D4×4)
3. **Weeks in Drought** = Consecutive weeks with D1+ drought
4. **Drought Trend** = Is drought worsening or improving?
5. **Severe Drought Area** = % of area in D2+ (critical threshold)
6. **Drought Duration** = How many months of sustained drought
7. **Seasonal Drought Pattern** = Drought level by season
8. **Multi-year Drought Indicator** = Drought persisting across years

#### Data Source Details:
- **Agency**: National Drought Mitigation Center (University of Nebraska-Lincoln)
- **URL**: [Drought Monitor Data](https://droughtmonitor.unl.edu/DmData/DataDownload.aspx)
- **Methodology**: Based on precipitation, temperature, soil moisture, streamflow, and vegetation health indices
- **Update Frequency**: Weekly (every Thursday)
- **Reliability**: Used by USDA, NOAA, and federal agencies for drought assessment


In [1]:
# Import libraries
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from pathlib import Path
import warnings
warnings.filterwarnings('ignore')

pd.set_option('display.max_columns', None)
sns.set_style('whitegrid')
plt.rcParams['figure.figsize'] = (14, 6)

print("✅ Libraries imported!")


✅ Libraries imported!


## 1. Load Drought Data


In [2]:
# Load drought data
drought_file = Path('../data/raw/drought/california_drought_2000_2025.csv')

print(f"Loading drought data from: {drought_file}")
print(f"File exists: {drought_file.exists()}")

drought_df = pd.read_csv(drought_file)
print(f"\n✅ Loaded {len(drought_df):,} weekly drought records!")
print(f"\nColumns: {drought_df.columns.tolist()}")
print(f"\nFirst 5 records:")
drought_df.head()


Loading drought data from: ../data/raw/drought/california_drought_2000_2025.csv
File exists: True

✅ Loaded 1,345 weekly drought records!

Columns: ['MapDate', 'StateAbbreviation', 'None', 'D0', 'D1', 'D2', 'D3', 'D4', 'ValidStart', 'ValidEnd', 'StatisticFormatID']

First 5 records:


,MapDate,StateAbbreviation,None,D0,D1,D2,D3,D4,ValidStart,ValidEnd,StatisticFormatID
0,20251007,CA,27.20,34.28,22.88,14.39,1.25,0.0,2025-10-07,2025-10-13,2
1,20250930,CA,26.78,34.70,19.91,17.36,1.25,0.0,2025-09-30,2025-10-06,2
2,20250923,CA,26.78,34.70,19.67,17.60,1.25,0.0,2025-09-23,2025-09-29,2
3,20250916,CA,26.78,33.67,16.55,19.51,3.49,0.0,2025-09-16,2025-09-22,2
4,20250909,CA,23.99,36.45,16.55,19.12,3.88,0.0,2025-09-09,2025-09-15,2
